In [2]:
# Importación de librerías

import pandas as pd
import numpy as np
import re
import xlsxwriter

In [3]:
ce1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\CE_1erSemestre.xlsx')
ce2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\CE_2dorSemestre.xlsx')
ptu1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\PTU_1erSemestre.xlsx')
ptu2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\PTU_2doSemestre.xlsx')
tns1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\TNS_1erSemestre.xlsx')
tns2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\TNS_2doSemestre.xlsx')
ce1_23 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\EstudiantesCobertura\[Evaluacion Docente de Estudiantes][02-06-2023].xlsx')

In [4]:
def clean_estudiantes(data):
    data = data.iloc[7:]
    data = data.iloc[:, 1:]
    data = data.reset_index(drop=True)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
    return data

In [5]:
ce1 = clean_estudiantes(ce1)
ce2 = clean_estudiantes(ce2)
ptu1 = clean_estudiantes(ptu1)
ptu2 = clean_estudiantes(ptu2)
tns1 = clean_estudiantes(tns1)
tns2 = clean_estudiantes(tns2)
ce1_23 =clean_estudiantes(ce1_23)


In [6]:

ce1_23 = ce1_23.rename(columns=lambda x: re.sub(r'^respuesta_1_1', 'consentimiento', x))
ce1_23 = ce1_23.rename(columns=lambda x: re.sub(r'^respuesta_2_', 'respuesta_1_', x))
ce1_23 = ce1_23.rename(columns=lambda x: re.sub(r'^respuesta_3_', 'respuesta_2_', x))

In [18]:
CoberturaEstudiantes = pd.concat([ce1,ce2,ptu1,ptu2,tns1,tns2,ce1_23])

In [19]:
CoberturaEstudiantes.loc[((CoberturaEstudiantes['estado'] == 'Finalizado') | (CoberturaEstudiantes['estado'] == 'Pendiente')) & (CoberturaEstudiantes['consentimiento'] == 'No Acepto participar. SALIR'), 'estado'] = 'No iniciada'

In [20]:
agrupacion_coberturas = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente','estado']

agrupacion_resultados = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente']

estados = ['finalizado','no_iniciada','pendiente']

In [21]:
ReporteCoberturas = CoberturaEstudiantes.groupby(agrupacion_coberturas).size().reset_index(name='count')

In [22]:
ReporteCoberturas = ReporteCoberturas.pivot_table(index=agrupacion_resultados, columns='estado',values='count',fill_value=0).reset_index().rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))

In [25]:
ReporteCoberturas['total']= ReporteCoberturas[estados].sum(axis=1)

In [28]:
ReporteCoberturas['finalizado_percent']= round((ReporteCoberturas['finalizado']/ReporteCoberturas['total'])*100,2)

In [30]:
ReporteCoberturas = ReporteCoberturas.reset_index(drop=True)

In [31]:
writer = pd.ExcelWriter('Resultados/ReporteCoberturasEstudiantes.xlsx', engine='xlsxwriter')
ReporteCoberturas.to_excel(writer, sheet_name='ReporteCoberturasEstudiantes', index=True)
writer.save()

C:\Users\aigna\AppData\Local\Temp\ipykernel_3848\965849539.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
